<a href="https://colab.research.google.com/github/aapoorv-tf/abs-summarizer-using-seq2seq/blob/main/seq2seq_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/gdrive/MyDrive/IR Project')

In [3]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [4]:
import tensorflow as tf
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')
 
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.92.3.210:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.92.3.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.92.3.210:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [5]:
!unzip "/content/gdrive/MyDrive/IR Project/Dataset/SmallDataset/news_summary.zip" -d "/content/gdrive/MyDrive/IR Project/Dataset/SmallDataset/"

Archive:  /content/gdrive/MyDrive/IR Project/Dataset/SmallDataset/news_summary.zip
  inflating: /content/gdrive/MyDrive/IR Project/Dataset/SmallDataset/news_summary.csv  
  inflating: /content/gdrive/MyDrive/IR Project/Dataset/SmallDataset/news_summary_more.csv  


In [5]:
d1 = pd.read_csv("/content/gdrive/MyDrive/IR Project/Dataset/SmallDataset/news_summary_more.csv", encoding='iso-8859-1')
d2 = pd.read_csv("/content/gdrive/MyDrive/IR Project/Dataset/SmallDataset/news_summary.csv", encoding='iso-8859-1')

In [6]:
d1.head()

,headlines,text
0,upGrad learner switches to career in ML & Al with 90% salary hike,"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program ..."
1,Delhi techie wins free food from Swiggy for one year on CRED,"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coi..."
2,New Zealand end Rohit Sharma-led India's 12-match winning streak,New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's capt...
3,Aegon life iTerm insurance plan helps customers save tax,"With Aegon Life iTerm Insurance plan, customers can enjoy tax benefits on your premiums paid and save up to Ã¢ÂÂ¹46,800^ on taxes. The plan provides life cover up to the age of 100 years. Also, c..."
4,"Have known Hirani for yrs, what if MeToo claims are not true: Sonam","Speaking about the sexual harassment allegations against Rajkumar Hirani, Sonam Kapoor said, ""I've known Hirani for many years...What if it's not true, the [#MeToo] movement will get derailed."" ""I..."


In [7]:
pre1 = d1.iloc[:, 0:2].copy()
pre2 = d2.iloc[:, 0:6].copy()

pre2['text'] = pre2['author'].str.cat(pre2['date'].str.cat(pre2['read_more'].str.cat(pre2['text'].str.cat(pre2['ctext'], sep=' '), sep=' '), sep=' '), sep=' ')

data = pd.DataFrame()
data['article'] = pd.concat([pre1['text'], pre2['text']], ignore_index=True)
data['summary'] = pd.concat([pre1['headlines'], pre2['headlines']],ignore_index=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102915 entries, 0 to 102914
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   article  102797 non-null  object
 1   summary  102915 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


In [9]:
data.head()

,article,summary
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program ...",upGrad learner switches to career in ML & Al with 90% salary hike
1,"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coi...",Delhi techie wins free food from Swiggy for one year on CRED
2,New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's capt...,New Zealand end Rohit Sharma-led India's 12-match winning streak
3,"With Aegon Life iTerm Insurance plan, customers can enjoy tax benefits on your premiums paid and save up to Ã¢ÂÂ¹46,800^ on taxes. The plan provides life cover up to the age of 100 years. Also, c...",Aegon life iTerm insurance plan helps customers save tax
4,"Speaking about the sexual harassment allegations against Rajkumar Hirani, Sonam Kapoor said, ""I've known Hirani for many years...What if it's not true, the [#MeToo] movement will get derailed."" ""I...","Have known Hirani for yrs, what if MeToo claims are not true: Sonam"


In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

stop_words = set(stopwords.words('english'))
def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    return (" ".join(tokens)).strip()

In [23]:
cleaned_article = []
data.article = data.article.astype(str)
for t in data['article']:
  cleaned_article.append(text_cleaner(t))

In [24]:
def summary_cleaner(text):
    newString = re.sub('"','', text)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    return newString

In [25]:
cleaned_summary = []
data.summary = data.summary.astype(str)
for t in data['summary']:
    cleaned_summary.append(str(summary_cleaner(t)))

In [28]:
cleaned_summary[1]

'delhi techie wins free food from swiggy for one year on cred'

In [29]:
data['cleaned_article'] = cleaned_article
data['cleaned_summary'] = cleaned_summary

In [30]:
wc = pd.DataFrame()
wc['article_count'] = data['cleaned_article'].apply(lambda x: len(str(x).split()))
for i in range(90,100):
  var = wc['article_count'].values
  var = np.sort(var,axis=None)
  print('At {} percentile, word count is {}'.format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile - ",var[-1])

At 90 percentile, word count is 41
At 91 percentile, word count is 41
At 92 percentile, word count is 42
At 93 percentile, word count is 42
At 94 percentile, word count is 43
At 95 percentile, word count is 45
At 96 percentile, word count is 108
At 97 percentile, word count is 173
At 98 percentile, word count is 227
At 99 percentile, word count is 300
100 percentile -  6737


In [31]:
wc = pd.DataFrame()
wc['summary_count'] = data['cleaned_summary'].apply(lambda x: len(str(x).split()))
for i in range(90,100):
  var = wc['summary_count'].values
  var = np.sort(var,axis=None)
  print('At {} percentile, word count is {}'.format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile - ",var[-1])

At 90 percentile, word count is 11
At 91 percentile, word count is 11
At 92 percentile, word count is 12
At 93 percentile, word count is 12
At 94 percentile, word count is 12
At 95 percentile, word count is 12
At 96 percentile, word count is 12
At 97 percentile, word count is 12
At 98 percentile, word count is 13
At 99 percentile, word count is 13
100 percentile -  17


In [32]:
max_len_art = 100
max_len_summ = 17

In [33]:
cleaned_article =np.array(data['cleaned_article'])
cleaned_summary=np.array(data['cleaned_summary'])

short_article=[]
short_summary=[]

for i in range(len(cleaned_article)):
    if(len(cleaned_summary[i].split())<=max_len_summ and len(cleaned_article[i].split())<=max_len_art):
        short_article.append(cleaned_article[i])
        short_summary.append(cleaned_summary[i])
        
df=pd.DataFrame({'article':short_article,'summary':short_summary})

In [34]:
df['summary'] = df['summary'].apply(lambda x: 'sostok '+ x +' eostok')

In [35]:
df.shape

(98737, 2)

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(np.array(df['article']),np.array(df['summary']),test_size=0.3,random_state=0,shuffle=True)

In [37]:
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_train))

#rare words coverage for article
thresh=4
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)


% of rare words in vocabulary: 57.26561672642552
Total Coverage of rare words: 2.334921930562173


In [38]:
x_tokenizer = Tokenizer(num_words = tot_cnt-cnt)
x_tokenizer.fit_on_texts(list(x_train))

x_train    =   x_tokenizer.texts_to_sequences(x_train) 
x_val   =   x_tokenizer.texts_to_sequences(x_val)

x_train    =   pad_sequences(x_train,  maxlen=max_len_art, padding='post') 
x_val   =   pad_sequences(x_val, maxlen=max_len_art, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [39]:
x_voc_size

66100

In [40]:
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_train))

# rare words coverage in summary
thresh=4
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 56.9882777276826
Total Coverage of rare words: 3.1210110019166093


In [41]:
y_tokenizer = Tokenizer(num_words = tot_cnt - cnt)
y_tokenizer.fit_on_texts(list(y_train))

y_train    =   y_tokenizer.texts_to_sequences(y_train) 
y_val   =   y_tokenizer.texts_to_sequences(y_val) 

y_train    =   pad_sequences(y_train, maxlen=max_len_summ, padding='post')
y_val   =   pad_sequences(y_val, maxlen=max_len_summ, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

In [42]:
y_voc_size

28835

In [43]:
y_tokenizer.word_counts['sostok'],len(y_train)

(69115, 69115)

In [44]:
#deleting empty rows
ind=[]
for i in range(len(y_train)):
    cnt=0
    for j in y_train[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_train=np.delete(y_train,ind, axis=0)
x_train=np.delete(x_train,ind, axis=0)

In [45]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [46]:
from keras import backend as K
K.clear_session()

latent_dim=300
embedding_dim=100

def seq2seq():
  
  global encoder_inputs
  global enc_emb
  global encoder_outputs
  global state_h
  global state_c
  global decoder_inputs
  global dec_emb_layer
  global decoder_lstm
  global decoder_dense
  encoder_inputs = Input(shape=(max_len_art,))
  #embedding layer
  
  enc_emb =  Embedding(x_voc_size, embedding_dim,trainable=True)(encoder_inputs)
  #encoder lstm 1
  encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
  encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

  #encoder lstm 2
  encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
  encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

    #encoder lstm 3
  encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
  
  encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)
  
  decoder_inputs = Input(shape=(None,))
  #embedding layer
  
  dec_emb_layer = Embedding(y_voc_size, embedding_dim,trainable=True)
  dec_emb = dec_emb_layer(decoder_inputs)

  
  decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
  decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

  #dense layer
  
  decoder_dense =  TimeDistributed(Dense(y_voc_size, activation='softmax'))
  decoder_outputs = decoder_dense(decoder_outputs)

  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  return model


In [47]:
with tpu_strategy.scope():
  model1=seq2seq()
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     6610000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100, 300), ( 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [48]:
from tensorflow.keras import optimizers
nadam = optimizers.Nadam(lr=0.0001,
                         beta_1=0.9,
                         beta_2=0.999,
                         epsilon=1e-8, 
                         schedule_decay=0.004)
model1.compile(loss='sparse_categorical_crossentropy',
              optimizer=nadam,
              metrics=['acc'])

In [49]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [51]:
history=model1.fit([x_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:],epochs=30,callbacks=[es],batch_size=256, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/30
270/270 [==============================] - 76s 211ms/step - loss: 5.4028 - acc: 0.3673 - val_loss: 4.8226 - val_acc: 0.3719
Epoch 2/30
270/270 [==============================] - 49s 183ms/step - loss: 4.7650 - acc: 0.3735 - val_loss: 4.6407 - val_acc: 0.3931
Epoch 3/30
270/270 [==============================] - 50s 183ms/step - loss: 4.5981 - acc: 0.4104 - val_loss: 4.4851 - val_acc: 0.4475
Epoch 4/30
270/270 [==============================] - 49s 183ms/step - loss: 4.4940 - acc: 0.4443 - val_loss: 4.4153 - val_acc: 0.4497
Epoch 5/30
270/270 [==============================] - 49s 183ms/step - loss: 4.4361 - acc: 0.4463 - val_loss: 4.3710 - val_acc: 0.4505
Epoch 6/30
270/270 [==============================] - 50s 183ms/step - loss: 4.3904 - acc: 0.4473 - val_loss: 4.3277 - val_acc: 0.4511
Epoch 7/30
270/270 [==============================] - 49s 183ms/step - loss: 4.3444 - acc: 0.4484 - val_loss: 4.2877 - val_acc: 0.4518
Epoch 8/30
270/270 [==============================] - 5

In [52]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [53]:

encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs,
                      state_h, state_c])


decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_hidden_state_input = Input(shape=(max_len_art, latent_dim))

dec_emb2 = dec_emb_layer(decoder_inputs)

(decoder_outputs2, state_h2, state_c2) = decoder_lstm(dec_emb2,
        initial_state=[decoder_state_input_h, decoder_state_input_c])

decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,
                      decoder_state_input_h, decoder_state_input_c],
                      [decoder_outputs2] + [state_h2, state_c2])

In [54]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_len_summ-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [55]:

def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString


In [62]:
for i in range(0,5):
  print("Article:",seq2text(x_train[i]))
  print("Original summary:",seq2summary(y_train[i]))
  print("Predicted summary:",decode_sequence(x_train[i].reshape(1,max_len_art)))
  print("\n")

Article: swadeshi jagran manch economic wing rss said would seek legislative amendments rules governing patents came supreme court tuesday ruled us seed maker monsanto claim patents genetically modified cotton seeds india need forbid companies charging patent amount added 
Original summary: rss outfit seeks change in patent rules post monsanto verdict 
Predicted summary:  pm modi to be to be to be to be to be to be


Article: retired australian spinner shane warne admitted ball century ashes fluke one deliveries leg spinners want bowl proud bowled warne said delivery warne first ashes pitched outside mike gatting leg stump spinning back hit stump 
Original summary: ball of the century was a admits shane warne 
Predicted summary:  kohli to be to be to be to be to be in india


Article: elon musk released video boring company first tunnel los angeles us construction almost complete musk said tunneling company offering free rides public months notably tunnel enable mass transit pedestrian